# HTML解析入门及准备URL生成连续技

*  本周主要内容：批量抓取页面基础及技巧


## 本周内容及学习目标

本周内容聚焦在

<mark> 如何有系统的把更多页数据(相同结构)作系统性爬取 </mark>

为此，我们需要学习

* 翻页：参数字典的拆解
  * xpath
  * 建构参数模板
  * 建构参数字典
* 翻页：系统性迭代
  * robots.txt
  * 频率及时间
* 翻页：数据备份与整合
  * 储存备份
  * 数据整合

In [1]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

## 0. 上周整合代码

In [36]:
# 上周C-1B-5 建构 参数模板  
参数_compTag_用户体验 = {'中国500强': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, '制造业500强': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, 'AI创新成长50强 ': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['189'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, '独角兽': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['130'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, '上市公司': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['156'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}}

# 上周C-1   多个页面准备测试1 中国500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['2018互联网300强']
r = session.get( url, params = payload)

# r.url

# 上周C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)


# 上周C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

# 上周C-4   输出
# df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

# 上周C-5 Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         5
经验         10
薪水         69
时间         24
职称        180
公司地点       68
公司名称       70
链结        190
公司URL      70
热门公司类型      6
dtype: int64


职称
公司名称            edu      
阿里巴巴            学历不限   45
小米              统招本科   12
明略科技集团          统招本科   11
华为              本科及以上  10
                统招本科    8
深圳市优必选科技股份有限公司  本科及以上   6
明略科技集团          本科及以上   5
新东方教育科技集团有限公司   统招本科    4
宁德时代新能源科技股份有限公司 大专及以上   4
滴滴              本科及以上   4
华为              硕士及以上   4
小米              本科及以上   4
远东控股集团          统招本科    4
达而观信息科技(上海)有限公司 统招本科    4
远东国际融资租赁有限公司    本科及以上   4
山东开创集团股份有限公司    大专及以上   3
小米              大专及以上   3
赛轮集团股份有限公司      本科及以上   3
旷视科技            本科及以上   3
SenseTime（商汤集团） 统招本科    3
通威股份            本科及以上   2
首约科技(北京)有限公司    本科及以上   2
新东方教育科技集团有限公司   本科及以上   2
阿里巴巴            统招本科    2
网易集团            本科及以上   2
朴新教育            统招本科    2
极智嘉科技           本科及以上   2
网易集团            大专及以上   2
深圳市优必选科技股份有限公司  硕士及以上   2
网易集团            统招本科    2
...                    ..
途虎养车网           本科及以上   1
车好多集团           大专及以上   1
宝德投资深圳          统招本科    1
爱回收             本科及以上   1
新城控股集团住宅开发事业部   本科及以上   1
小红书             统招本科    1
山东荣信集团有限公司      本科及以上   1
岁宝百货            大专及以上   1
                本科及以上   1
平安医疗健康管理股份有限公司  本科及以上   1
平安银行            本科及以上   1
戴维医疗            统招本科    1
招金矿业            统招本科    1
安徽广信农化股份有限公司    统招本科    1
Henlius         本科及以上   1
新城悦控股有限公司       大专及以上   1
方太              本科及以上   1
宝德投资深圳          大专及以上   1
方太              统招本科    1
太平洋建设           本科及以上   1
天际电器            大专及以上   1
天士力集团网          统招本科    1
合生创展集团有限公司      大专及以上   1
欧菲光             大专及以上   1
                本科及以上   1
博世              统招本科    1
深圳市优必选科技股份有限公司  统招本科    1
深圳市农产品集团股份有限公司  本科及以上   1
深圳市客路网络科技有限公司   本科及以上   1
深圳越海全球供应链有限公司   学历不限    1

[93 rows x 1 columns]

# 翻页：参数字典的拆解
## xpath解析翻页a/@href

In [2]:
# A-0   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

In [4]:
# A-1  xpath 解析翻页a/@href
# xpath_翻页a = '//div[@class="pagerbar"]/a' # 有disabled, current等href是javascript
# 新方法判别式，starts-with是一个函数，抓取"zhaopin"开头的超链接
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'
# print (r.html.xpath(xpath_翻页a)) # 物件

href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
#print (href_列表)

文字_列表 = [x.text for x in r.html.xpath(xpath_翻页a)]
#print (文字_列表)

# 将页码和页码链接利用字典推导式存进字典里
# 对比href_列表和文字_列表输出的结果，href_字典相当于拼接href_列表和文字_列表成字典，使各自有对应的值
href_字典 = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
print (href_字典)

{'2': '/zhaopin/?init=-1&headckid=9134373743c694c4&fromSearchBtn=2&keyword=PRD&ckid=9134373743c694c4°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=19527483055193b9b8437fe6af6670d6&d_curPage=0&d_pageSize=40&d_headId=19527483055193b9b8437fe6af6670d6&curPage=1', '3': '/zhaopin/?init=-1&headckid=9134373743c694c4&fromSearchBtn=2&keyword=PRD&ckid=9134373743c694c4°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=19527483055193b9b8437fe6af6670d6&d_curPage=0&d_pageSize=40&d_headId=19527483055193b9b8437fe6af6670d6&curPage=2', '4': '/zhaopin/?init=-1&headckid=9134373743c694c4&fromSearchBtn=2&keyword=PRD&ckid=9134373743c694c4°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=19527483055193b9b8437fe6af6670d6&d_curPage=0&d_pageSize=40&d_headId=19527483055193b9b8437fe6af6670d6&curPage=3', '5': '/zhaopin/?init=-1&headckid=9134373743c694c4&fromSearchBtn=2&keyword=P

### 观察：
此网页是否给出开始丶步进丶及结束的信息，以方便我们完成迭代设置

## 建构参数模板

In [6]:
# A-2 建构参数模板：找到关键参数及参数结构

# 需要模组库
from urllib.parse import urlparse, parse_qs
import pandas as pd
from IPython.display import display, HTML

# 总体目标：输入 href_列表, 建构出参数字典

# urlparse 解析后丢入数据框
# 对href列表里的链接进行解析
df = pd.DataFrame([ urlparse(x) for x in href_列表])  
# 解析query，返回词典格式数据，词典的key是query中变量名字，value是对应的值。
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])

display(df)
print(df.nunique())
display(df_qs)
print(df_qs.nunique())

df_qs.curPage
df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) # astype(int)变成整数
df_qs

,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=9134373743c694c4&fromSearchBt...,
1,,,/zhaopin/,,init=-1&headckid=9134373743c694c4&fromSearchBt...,
2,,,/zhaopin/,,init=-1&headckid=9134373743c694c4&fromSearchBt...,
3,,,/zhaopin/,,init=-1&headckid=9134373743c694c4&fromSearchBt...,
4,,,/zhaopin/,,init=-1&headckid=9134373743c694c4&fromSearchBt...,
5,,,/zhaopin/,,init=-1&headckid=9134373743c694c4&fromSearchBt...,


scheme      1
netloc      1
path        1
params      1
query       5
fragment    1
dtype: int64


,init,headckid,fromSearchBtn,keyword,ckid,siTag,d_sfrom,d_ckId,d_curPage,d_pageSize,d_headId,curPage
0,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,1
1,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,2
2,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,3
3,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,4
4,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,1
5,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,9


init             1
headckid         1
fromSearchBtn    1
keyword          1
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
curPage          5
dtype: int64


,init,headckid,fromSearchBtn,keyword,ckid,siTag,d_sfrom,d_ckId,d_curPage,d_pageSize,d_headId,curPage,curPage_int
0,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,1,1
1,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,2,2
2,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,3,3
3,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,4,4
4,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,1,1
5,-1,9134373743c694c4,2,PRD,9134373743c694c4°radeFlag=0,1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw,search_unknown,19527483055193b9b8437fe6af6670d6,0,40,19527483055193b9b8437fe6af6670d6,9,9


### 观察：
* query
* curPage 5次, 最大值9, 本页不算?

-----

## 建构参数模板：curPage


In [7]:
# A-2 建构参数模板：找到关键参数及参数结构

def parse_url_qs_for_curPage (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# 取一例做模板
参数模板 = parse_url_qs_for_curPage(href_列表[0])
print (参数模板)
# 观察参数模板，可以找到我们的关键参数是PRD，可以观察到这一页有多少个数据
print (href_字典)

{'init': ['-1'], 'headckid': ['9134373743c694c4'], 'fromSearchBtn': ['2'], 'keyword': ['PRD'], 'ckid': ['9134373743c694c4°radeFlag=0'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['19527483055193b9b8437fe6af6670d6'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['19527483055193b9b8437fe6af6670d6'], 'curPage': ['1']}
{'2': '/zhaopin/?init=-1&headckid=9134373743c694c4&fromSearchBtn=2&keyword=PRD&ckid=9134373743c694c4°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=19527483055193b9b8437fe6af6670d6&d_curPage=0&d_pageSize=40&d_headId=19527483055193b9b8437fe6af6670d6&curPage=1', '3': '/zhaopin/?init=-1&headckid=9134373743c694c4&fromSearchBtn=2&keyword=PRD&ckid=9134373743c694c4°radeFlag=0&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=19527483055193b9b8437fe6af6670d6&d_curPage=0&d_pageSize=40&d_headId=19527483055193b9b8437fe6af6670d6&curPage=2'

In [9]:
# A-3 建构参数模板生成器：keyword curPage
def 参数模板生成(keyword, curPage):
    参数 = 参数模板.copy()
    参数['curPage'] = curPage
    参数['keyword'] = keyword
    return (参数)

参数_keyword_用户体验_curPage = { i:参数模板生成(curPage = [i],keyword = ['用户体验']) for i,v in href_字典.items() }

# print(参数_keyword_用户体验_curPage) # 只生成本页有的额外翻页URL, 并没有推估到&curPage=9,也没有这页

print (df_qs.curPage_int.min()) # 最小值只有1
print (df_qs.curPage_int.max()) # 最大值只有9

# 应该是 0 (本页)....9(最大值)

参数_keyword_用户体验_curPage = { i:参数模板生成(curPage = [i],keyword = ['用户体验']) for i in range(0,df_qs.curPage_int.max()+1) } # 从最小值遍历到最大值
参数_keyword_用户体验_curPage

1
9


{0: {'init': ['-1'],
  'headckid': ['9134373743c694c4'],
  'fromSearchBtn': ['2'],
  'keyword': ['用户体验'],
  'ckid': ['9134373743c694c4°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['19527483055193b9b8437fe6af6670d6'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['19527483055193b9b8437fe6af6670d6'],
  'curPage': [0]},
 1: {'init': ['-1'],
  'headckid': ['9134373743c694c4'],
  'fromSearchBtn': ['2'],
  'keyword': ['用户体验'],
  'ckid': ['9134373743c694c4°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['19527483055193b9b8437fe6af6670d6'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['19527483055193b9b8437fe6af6670d6'],
  'curPage': [1]},
 2: {'init': ['-1'],
  'headckid': ['9134373743c694c4'],
  'fromSearchBtn': ['2'],
  'keyword': ['用户体验'],
  'ckid': ['9134373743c694c4°radeFlag=0'],
  'siTag': ['1B2M2Y8AsgTpgAmY

# 翻页：系统性迭代

## 爬亦有道
* robots.txt 站长/网站拥有者给搜索引擎的"道"
* 频率及时间
  * 不要爬太快
  * 尽量像"人"一样礼貌
  * time.sleep

In [10]:
# B-1 上周C-2  简化 上上周A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)


## 爬亦有道- 不要爬太快
time.sleep

In [11]:
import time
from random import random
time.sleep(3+4*random())
# 取平均值，如上所示，平均约5秒
# 不能怕太快的噢，否则有可能会被封ip

In [33]:
# B-2 多个页面，单个关键词
# 放慢脚步 time.sleep
# %%time 把魔术命令注释了，有时运行会报错而有时又可以顺利运行

import time
from random import random

url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_keyword_用户体验_curPage.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
    df = df.assign (curPage = k)  # 区分  curPage
    list_df.append(df)

df_all = pd.concat(list_df).reset_index() # pd.concat拼接数据；reset_index可以还原索引，重新变为默认的整型索引 
df_all.index.name = '序'

# 上周C-4   输出
df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页.xlsx",sheet_name="用户体验")

# 预估时间: 5秒*10 =50
# 预估数量: 40*10 =400

In [34]:
## 多个页面+多个关键词
time.sleep

<function time.sleep>

In [35]:
%%time
# B-3 多个页面+多个关键词
import time
from random import random

url = "https://www.liepin.com/zhaopin/"
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'

keywords = ['用户体验','UX']
list_df = list()

## 第一页试探有多长的页面
for key in keywords:
    payload = 参数模板生成(keyword=[key], curPage=['0'])
    df = requests_liepin( url, params = payload)
    href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
    df = pd.DataFrame([ urlparse(x) for x in href_列表])
    df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
    df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) # 变成整数
    长度 = df_qs.curPage_int.max()+1
    参数_keyword_X_curPage = { i:参数模板生成(curPage = [i],keyword = [key]) for i in range(0,长度) }
    #print (参数_keyword_X_curPage)
    print (key,长度)
    
    for k,v in 参数_keyword_X_curPage.items():
        payload = v
        df = requests_liepin( url, params = payload)
        time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
        df = df.assign (keyword = key)  # 区分  keyword    assign()直接在表格最后加多一列
        df = df.assign (curPage = k)  # 区分  curPage    
        list_df.append(df)
        
df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页.xlsx",sheet_name="_".join(keywords))
# 预估时间: 2*5秒*10 =100
# 预估数量: 2*40*10 =800

用户体验 10
UX 10
Wall time: 2min 3s


# 翻页：数据备份与整合
多个页面+多个关键词执行时，若怕中断最好把每一页的df内容备份做中继

In [38]:
%%time
# C-1 多个页面+多个关键词
import time
from random import random

url = "https://www.liepin.com/zhaopin/"
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'

keywords = ['用户体验','UX','产品需求','PRD']
list_df = list()

## 第一页试探有多长的页面
for key in keywords:
    payload = 参数模板生成(keyword=[key], curPage=['0'])
    df = requests_liepin( url, params = payload)
    href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
    df = pd.DataFrame([ urlparse(x) for x in href_列表])
    df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
    df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) # 变成整数
    长度 = df_qs.curPage_int.max()+1
    参数_keyword_X_curPage = { i:参数模板生成(curPage = [i],keyword = [key]) for i in range(0,长度) }
    #print (参数_keyword_X_curPage)
    print (key,长度)
    
    for k,v in 参数_keyword_X_curPage.items():
        payload = v
        df = requests_liepin( url, params = payload)
        time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
        ## 备份
        df.to_csv("20春_Web数据挖掘_week04_liepin_{key}_{k}.tsv"\
                  .format(key=key, k=k), sep="\t", encoding="utf8")
        
        df = df.assign (keyword = key)  # 区分  keyword    
        df = df.assign (curPage = k)  # 区分  curPage    
        list_df.append(df)
        
df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("20春_Web数据挖掘_week04_liepin_翻页_4.xlsx",\
                sheet_name="_".join(keywords))
# 预估时间: 4*5秒*10 =200
# 预估数量: 4*40*10 =1600

用户体验 10
UX 10
产品需求 10
PRD 10
Wall time: 4min 16s


# 本周练习
尝试爬取多个页面一个关键词


In [47]:
from urllib.parse import urlparse, parse_qs
import pandas as pd
from IPython.display import display, HTML
import time
from random import random

# xpath 翻页
url = "https://www.liepin.com/zhaopin/?keyword=Python"
session = HTMLSession()
r = session.get( url )

xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'
# print (r.html.xpath(xpath_翻页a)) # 物件

href_list = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
#print (href_列表)

文字_list = [x.text for x in r.html.xpath(xpath_翻页a)]
#print (文字_列表)

# 将页码和页码链接利用字典推导式存进字典里
href_dict = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
# print (href_dict)

# 解析ur参数以及query
df = pd.DataFrame([ urlparse(x) for x in href_list])  
# 解析query，返回词典格式数据，词典的key是query中变量名字，value是对应的值。
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])

# display(df)
print(df.nunique())
# display(df_qs)
print(df_qs.nunique())

df_qs.curPage
df_qs = df_qs.assign (curPage_int=df_qs.curPage.astype(int)) 

# 构建参数模板
def parse_url_qs_for_curPage (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# 取一例做模板
参数模板 = parse_url_qs_for_curPage(href_list[0])
print (参数模板)

# 构建参数字典
def 参数模板生成(keyword, curPage):
    参数 = 参数模板.copy()
    参数['curPage'] = curPage
    参数['keyword'] = keyword
    return (参数)

参数_keyword_用户体验_curPage = { 
    i:参数模板生成(curPage = [i],keyword = ['用户体验']) \
    for i,v in href_字典.items()\
    }
print (df_qs.curPage_int.min()) 
print (df_qs.curPage_int.max())

参数_keyword_用户体验_curPage = { 
    i:参数模板生成(curPage = [i],keyword = ['用户体验']) for i in range(0,df_qs.curPage_int.max()+1) } # 从最小值遍历到最大值
参数_keyword_用户体验_curPage

# 输出多个页面一个关键词的数据
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_keyword_用户体验_curPage.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())  
    df = df.assign (curPage = k)  
    list_df.append(df)

df_all = pd.concat(list_df).reset_index() 
df_all.index.name = '序'

# 输出
df_all.to_excel("week04_test.xlsx",sheet_name="用户体验")



scheme      1
netloc      1
path        1
params      1
query       5
fragment    1
dtype: int64
ckid             1
curPage          5
d_ckId           1
d_curPage        1
d_headId         1
d_pageSize       1
d_sfrom          1
fromSearchBtn    1
headckid         1
init             1
keyword          1
siTag            1
dtype: int64
{'init': ['-1'], 'headckid': ['4cd9cc81e0e2b5bb'], 'fromSearchBtn': ['2'], 'keyword': ['Python'], 'ckid': ['4cd9cc81e0e2b5bb°radeFlag=0'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f66fbdc1a69f177eaed041d2048e1d6f'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f66fbdc1a69f177eaed041d2048e1d6f'], 'curPage': ['1']}
1
9
